In [60]:
%reset -f
%matplotlib inline

In [61]:
import numpy as np
import scipy.io as sio
import sklearn as sk
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [62]:
data = sio.loadmat('/Users/jeff/Documents/Python/_projects/cyclingRNN/drakeFeb.mat')

In [63]:
m1 = np.transpose(data['D'][0,0]['M1'], [1, 0, 2, 3])
emg = np.transpose(data['D'][0,0]['EMG'], [1, 0, 2, 3])

print 'M1 shape '+str(m1.shape)
print 'EMG shape '+str(emg.shape)

M1 shape (5092, 113, 2, 2)
EMG shape (5092, 36, 2, 2)


RNN

In [64]:
from tensorflow.models.rnn import rnn, rnn_cell

In [106]:
# Some batch-related functions
def hankelize(a, num_steps_):
    """
    Convert size (T,n) matrix to size (num_steps, T-num_steps, n) tensor.
    Different batches can now be selected by a[:,batch_inds,:].
    (Hankelize is not the correct term).
    """
    dims = a.shape
    T_new = dims[0] - num_steps_
    b = np.zeros((T_new, num_steps_) + dims[1:])
    for i in range(T_new):
        b[i,:,:] = a[i:i+num_steps_, :]
    inds = np.arange(a.ndim)
    inds[0], inds[1] = inds[1], inds[0]
    b = np.transpose(b, inds) # |
    return b

def get_rand_batch(a, batch_size_):
    """
    args: 
        a: size (num_steps, T_new, n) array
        batch_size: how many batches to select
    return:
        inds: indices for initial states of each batch
        a: size (num_steps, batch_size, n) array
    """
    inds = np.random.choice(T_new, batch_size_)    
    for i in range(len(a)):
        a[i] = a[i][:,inds,:] 
    return inds, a

In [100]:
# Time and batch parameters
num_steps = 30 
batch_size = 50
T_new = emg.shape[1] - num_steps

In [101]:
y_data = hankelize(emg, 10)

In [105]:
y_data.transpose([1,0,2,3,4]).shape

(10, 5082, 36, 2, 2)

### graph

In [108]:
tf.reset_default_graph()
m = 0
n = 100
p = y_data.shape[1]

In [ ]:
X = tf.Variable()